# Superheated Rankine cycle

---

Consider a steam power plant operating on the simple ideal Rankine cycle.
Steam enters the turbine at **3 MPa** and **350 °C** and is condensed in the condenser at a pressure of **75 kPa**.

<img align="left" src='./assets/images/Rankine_1.png' />
<img align="right" src='./assets/images/Rankine_2.png' />

In [2]:
import cantera as ct

### Defining solutions and setting initial conditions

In [3]:
p1 = 75 * 1e3
p2 = 3 * 1e6
t3 = 350 + 273.15

## Actual vapor power cycle vs ideal Rankine cycle

<img align="right" src='./assets/images/Rankine_3.png' />

- FLuid friction --> pressure drop
- Heat losses
- Pump and turbine irreversibilities 



<img align="left" src='./assets/images/Rankine_4.png' style="width:500px;margin-right:10px;"/>

Consider a steam power plant operating as for the previous example.
If the isentropic efficiency of the **turbine** is **87 %** and the isentropic efficiency of the **pump** is **85 %**, how does this affect the process?

**Note**: starting from the previous example, you should be implement the isentropic efficiencies in the "pump" and "expand" functions and solve the exercise.

$$ \eta_P = \frac{W_s}{W_a} = \frac{h_{2s} - h_1}{h_2 - h_1} $$
$$ \eta_T = \frac{W_a}{W_s} = \frac{h_3 - h_4}{h_3 - h_{4s}}$$

In [4]:
eta_pump = 0.85  # pump isentropic efficiency
eta_turbine = 0.87  # turbine isentropic efficiency

### Funtions

In [5]:
def pump(fluid, p_final: float, eta: float) -> float:
    """Adiabatically pump a fluid to pressure p_final, using
    a pump with isentropic efficiency eta."""
    h0 = fluid.h
    s0 = fluid.s
    fluid.SP = s0, p_final
    h1s = fluid.h
    isentropic_work = h1s - h0
    actual_work = isentropic_work / eta
    h1 = h0 + actual_work
    fluid.HP = h1, p_final
    s1 = fluid.s
    return actual_work

In [6]:
def expand(fluid, p_final: float, eta: float) -> float:
    """Adiabatically expand a fluid to pressure p_final, using
    a turbine with isentropic efficiency eta."""
    h0 = fluid.h
    s0 = fluid.s
    fluid.SP = s0, p_final
    h1s = fluid.h
    isentropic_work = h0 - h1s
    actual_work = isentropic_work * eta
    h1 = h0 - actual_work
    fluid.HP = h1, p_final
    return actual_work

In [7]:
def printState(n, fluid):
    print("\n***************** State {0} ******************".format(n))
    print(fluid.report())

### Create an object representing water

In [8]:
w = ct.Water()

### Start with saturated liquid water at p1

In [11]:
w.PQ = p1, 0.0
h1 = w.h
p1 = w.P
printState(1, w)


***************** State 1 ******************

  water:

       temperature   364.96 K
          pressure   75000 Pa
           density   964.01 kg/m^3
  mean mol. weight   18.016 kg/kmol
    vapor fraction   0
   phase of matter   liquid-gas-mix

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -1.5586e+07        -2.808e+08  J
   internal energy       -1.5586e+07        -2.808e+08  J
           entropy            4733.3             85276  J/K
    Gibbs function       -1.7314e+07       -3.1192e+08  J
 heat capacity c_p            4206.5             75784  J/K
 heat capacity c_v            3814.8             68728  J/K



### Pump it adiabatically to p_max

In [9]:
pump_work = pump(w, p2, eta_pump)
h2 = w.h
printState(2, w)


***************** State 2 ******************

  water:

       temperature   365.27 K
          pressure   3e+06 Pa
           density   965.15 kg/m^3
  mean mol. weight   18.016 kg/kmol
    vapor fraction   0
   phase of matter   liquid

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -1.5583e+07       -2.8074e+08  J
   internal energy       -1.5586e+07       -2.8079e+08  J
           entropy            4734.8             85302  J/K
    Gibbs function       -1.7312e+07        -3.119e+08  J
 heat capacity c_p            4200.2             75671  J/K
 heat capacity c_v            3808.5             68613  J/K



### Heat it to t3

In [10]:
w.TP = t3, p2
h3 = w.h
heat_added = h3 - h2
printState(3, w)


***************** State 3 ******************

  water:

       temperature   623.15 K
          pressure   3e+06 Pa
           density   11.046 kg/m^3
  mean mol. weight   18.016 kg/kmol
    vapor fraction   1
   phase of matter   gas

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -1.2856e+07       -2.3161e+08  J
   internal energy       -1.3127e+07        -2.365e+08  J
           entropy             10263        1.8489e+05  J/K
    Gibbs function       -1.9251e+07       -3.4682e+08  J
 heat capacity c_p              2355             42428  J/K
 heat capacity c_v            1718.6             30962  J/K



### Expand back to p1

In [11]:
turbine_work = expand(w, p1, eta_turbine)
x4 = w.Q
printState(4, w)


***************** State 4 ******************

  water:

       temperature   364.96 K
          pressure   75000 Pa
           density   0.48684 kg/m^3
  mean mol. weight   18.016 kg/kmol
    vapor fraction   0.92633
   phase of matter   liquid-gas-mix

                          1 kg             1 kmol     
                     ---------------   ---------------
          enthalpy       -1.3476e+07       -2.4278e+08  J
   internal energy        -1.363e+07       -2.4555e+08  J
           entropy             10517        1.8947e+05  J/K
    Gibbs function       -1.7314e+07       -3.1192e+08  J
 heat capacity c_p   inf               inf              J/K
 heat capacity c_v   nan               nan              J/K



### Efficiency

In [12]:
eff = (turbine_work - pump_work) / heat_added

print("efficiency = ", eff)

efficiency =  0.22606542015829068
